## Hometask

- ### Mine top 10 association rules worth recommending for Groceries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from apyori import apriori


In [ ]:
path = '/Users/vladislavpleshko/Documents/VS Code/before/amazinum/data/Groceries_dataset.csv'
df = pd.read_csv(path)
print(df.isnull().sum())
df


Member_number      0
Date               0
itemDescription    0
dtype: int64


,Member_number,Date,itemDescription
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk
...,...,...,...
38760,4471,08-10-2014,sliced cheese
38761,2022,23-02-2014,candy
38762,1097,16-04-2014,cake bar
38763,1510,03-12-2014,fruit/vegetable juice


In [ ]:
help(apriori)


Help on function apriori in module apyori:

apriori(transactions, **kwargs)
    Executes Apriori algorithm and returns a RelationRecord generator.

    Arguments:
        transactions -- A transaction iterable object
                        (eg. [['A', 'B'], ['B', 'C']]).

    Keyword arguments:
        min_support -- The minimum support of relations (float).
        min_confidence -- The minimum confidence of relations (float).
        min_lift -- The minimum lift of relations (float).
        max_length -- The maximum length of the relation (integer).



In [ ]:
# Для кожного рядкаd split по '/' і очистимо пробіли
df['ItemList'] = df['itemDescription'].apply(
    lambda cell: [item.strip() for item in cell.split('/') if item.strip()]
)

transactions = df['ItemList'].tolist()

transactions


[['tropical fruit'],
 ['whole milk'],
 ['pip fruit'],
 ['other vegetables'],
 ['whole milk'],
 ['rolls', 'buns'],
 ['other vegetables'],
 ['pot plants'],
 ['whole milk'],
 ['tropical fruit'],
 ['citrus fruit'],
 ['beef'],
 ['frankfurter'],
 ['chicken'],
 ['butter'],
 ['fruit', 'vegetable juice'],
 ['packaged fruit', 'vegetables'],
 ['chocolate'],
 ['specialty bar'],
 ['other vegetables'],
 ['butter milk'],
 ['whole milk'],
 ['tropical fruit'],
 ['tropical fruit'],
 ['bottled water'],
 ['yogurt'],
 ['sausage'],
 ['other vegetables'],
 ['brown bread'],
 ['yogurt'],
 ['hamburger meat'],
 ['root vegetables'],
 ['pork'],
 ['beef'],
 ['pastry'],
 ['fruit', 'vegetable juice'],
 ['canned beer'],
 ['root vegetables'],
 ['citrus fruit'],
 ['sausage'],
 ['tropical fruit'],
 ['berries'],
 ['canned beer'],
 ['butter milk'],
 ['coffee'],
 ['pastry'],
 ['rolls', 'buns'],
 ['misc. beverages'],
 ['root vegetables'],
 ['sausage'],
 ['canned beer'],
 ['ham'],
 ['turkey'],
 ['whole milk'],
 ['whole milk']

In [148]:
rules = apriori(
    transactions, min_support=0.001, min_confidence=0.2, min_lift=1.2, min_length=2
)
records = []
for rule in rules:
    support = rule.support
    for stat in rule.ordered_statistics:
        base = tuple(stat.items_base)
        add = tuple(stat.items_add)
        confidence = stat.confidence
        lift = stat.lift
        print(f"Rule: {base} -> {add}")
        print(f"  Support: {support:.3f}")
        print(f"  Confidence: {confidence:.3f}")
        print(f"  Lift: {lift:.3f}")
        print("-" * 40)
        records.append({
            'base': base,
            'add': add,
            'support': support,
            'confidence': confidence,
            'lift': lift
        })
df_rules = pd.DataFrame.from_records(records)

# костильне рішення, але все ж робоче
# мені здається немає сенсу відображатись два правила по одній транзакції, так як значення однакові
mask = ~df_rules['lift'].duplicated(keep='first')
result = df_rules[mask].sort_values(by='lift', ascending=False)
result


Rule: ('bags',) -> ('cling film',)
  Support: 0.002
  Confidence: 0.949
  Lift: 496.987
----------------------------------------
Rule: ('cling film',) -> ('bags',)
  Support: 0.002
  Confidence: 1.000
  Lift: 496.987
----------------------------------------
Rule: ('blush wine',) -> ('red',)
  Support: 0.004
  Confidence: 1.000
  Lift: 246.911
----------------------------------------
Rule: ('red',) -> ('blush wine',)
  Support: 0.004
  Confidence: 1.000
  Lift: 246.911
----------------------------------------
Rule: ('buns',) -> ('rolls',)
  Support: 0.044
  Confidence: 1.000
  Lift: 22.590
----------------------------------------
Rule: ('rolls',) -> ('buns',)
  Support: 0.044
  Confidence: 1.000
  Lift: 22.590
----------------------------------------
Rule: ('film',) -> ('photo',)
  Support: 0.002
  Confidence: 1.000
  Lift: 490.696
----------------------------------------
Rule: ('photo',) -> ('film',)
  Support: 0.002
  Confidence: 1.000
  Lift: 490.696
---------------------------------

,base,add,support,confidence,lift
0,"(bags,)","(cling film,)",0.001909,0.948718,496.987179
6,"(film,)","(photo,)",0.002038,1.000000,490.696203
10,"(packaged fruit,)","(vegetables,)",0.003302,1.000000,302.851562
2,"(blush wine,)","(red,)",0.004050,1.000000,246.910828
8,"(fruit,)","(vegetable juice,)",0.013363,1.000000,74.835907
12,"(sour cream,)","(whipped,)",0.017077,1.000000,58.557402
4,"(buns,)","(rolls,)",0.044267,1.000000,22.590326


In [145]:
"""" Хотілося б піти далі та заглибитись у задачу, отримавши об’єктивний score,
за яким могли б відсортувати правила й мати чітке бачення їхньої цінності.
Саме тому я додав нормалізацію метрик (support, confidence, lift) та поєднав їх
в один багатокритеріальний показник. Не знаю чи це практикується у реальних задачах, але логіка цікава.
"""

alpha, beta, gamma = 1, 1, 1 # ваги

df_rules['norm_support'] = (df_rules['support'] - df_rules['support'].min()) / (df_rules['support'].max() - df_rules['support'].min())
df_rules['norm_confidence'] = (df_rules['confidence'] - df_rules['confidence'].min()) / (df_rules['confidence'].max() - df_rules['support'].min())
df_rules['norm_lift'] = (df_rules['lift'] - df_rules['lift'].min()) / (df_rules['lift'].max() - df_rules['lift'].min())

df_rules['score'] = (
    alpha * df_rules['norm_support']
    + beta * df_rules['norm_confidence']
    + gamma * df_rules['norm_lift']
)


In [146]:
# top 10 rules (based on 'score')

filtered = df_rules[mask].sort_values(by='score', ascending=False).reset_index(drop=True)
filtered.head(10)


,base,add,support,confidence,lift,norm_support,norm_confidence,norm_lift,score
0,"(buns,)","(rolls,)",0.044267,1.000000,22.590326,1.000000,0.05138,0.000000,1.051380
1,"(film,)","(photo,)",0.002038,1.000000,490.696203,0.003045,0.05138,0.986739,1.041164
2,"(bags,)","(cling film,)",0.001909,0.948718,496.987179,0.000000,0.00000,1.000000,1.000000
3,"(packaged fruit,)","(vegetables,)",0.003302,1.000000,302.851562,0.032887,0.05138,0.590774,0.675041
4,"(blush wine,)","(red,)",0.004050,1.000000,246.910828,0.050548,0.05138,0.472854,0.574782
5,"(sour cream,)","(whipped,)",0.017077,1.000000,58.557402,0.358100,0.05138,0.075816,0.485296
6,"(fruit,)","(vegetable juice,)",0.013363,1.000000,74.835907,0.270402,0.05138,0.110131,0.431913
